### Will train our LivenessNet classifier. Used TensorFlow to train the model.
process results in a few files:
    - le.pickle : Our class label encoder.
    - liveness.model : Our serialized Tensorflow model which detects face liveness.
    - plot.png : The training history plot shows accuracy and loss curves so we can assess our model 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from pyimagesearch.livenessnet import LivenessNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [3]:
from tensorflow.keras.models import model_from_json
# model.load_weights('vgg_face_weights.h5')

In [4]:
args = {
    'dataset': 'dataset', # path to input dataset
    'model': 'liveness.model', # path to trained model
    'le': 'le.pickle', # path to label encoder
    'plot':'plot.png', # output loss/accuracy plot 
}

In [5]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 64
EPOCHS = 101

In [6]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] Get list of images")
imagePaths = list(paths.list_images(args["dataset"]))
print('imagePaths len: ', len(imagePaths))
data = []
labels = []
print("[INFO] Done.")


[INFO] Get list of images
imagePaths len:  1464
[INFO] Done.


In [7]:
# loop over all image paths
print("[INFO] loading images...")
for imagePath in imagePaths:
    # extract the class label from the filename, load the image and
    # resize it to be a fixed 32x32 pixels, ignoring aspect ratio
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
    
print("[INFO] done loading images...")

[INFO] loading images...
[INFO] done loading images...


In [8]:
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0 # normalizing images

In [9]:
# encode the labels (which are currently strings) as integers and then one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

In [10]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [11]:
# construct the training image generator with data augmentation
aug = ImageDataGenerator(rotation_range=20, 
                         zoom_range=0.15, 
                         width_shift_range=0.2, 
                         height_shift_range=0.2, 
                         shear_range=0.15,
                         horizontal_flip=True, 
                         fill_mode="nearest")

In [12]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model = LivenessNet.build(width=32, height=32, depth=3, classes=len(le.classes_))

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(x=aug.flow(trainX, 
                         trainY, 
                         batch_size=BS), 
                         validation_data=(testX, testY), 
                         steps_per_epoch=len(trainX) // BS,
                         epochs=EPOCHS,
                         verbose=0)

[INFO] training network for 101 epochs...


In [14]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

        fake       0.47      0.37      0.41       181
        real       0.49      0.59      0.53       185

    accuracy                           0.48       366
   macro avg       0.48      0.48      0.47       366
weighted avg       0.48      0.48      0.47       366



In [15]:
# save the network to disk
print("[INFO] serializing network to '{}'...".format(args["model"]))
model.save(args["model"], save_format="h5")

[INFO] serializing network to 'liveness.model'...


In [16]:
# save the label encoder to disk
f = open(args["le"], "wb")
f.write(pickle.dumps(le))
f.close()

In [17]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])